In [8]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {"영화":"라이프 오브 파이","answer":"🐅🌊🚣"},
    {"영화":"인터스텔라","answer":"🚀🌌⏰"},
    {"영화":"기생충","answer":"🏠💰🕷️"},
    {"영화":"타이타닉","answer":"🚢💎💕"},
]

# Few-shot 예시를 위한 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "영화: {영화}"),
    ("ai", "{answer}")
])

# Few-shot 프롬프트 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    input_variables=["movie_name"]
)

memory = ConversationSummaryBufferMemory(llm=chat, max_token_limit=120, return_messages=True)

# 최종 프롬프트 템플릿
final_prompt = ChatPromptTemplate.from_messages([
   ("system", """You are a helpful AI assistant that specializes in movies and emojis. 

When the user asks for a movie name to be converted to emojis, respond with exactly 3 emojis that best represent the movie.


Always be helpful and informative in your responses."""),
    few_shot_prompt,
    MessagesPlaceholder(variable_name="history"),
    ("human", "{movie_name}")
])

def load_memory(input_dict):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | chat

def invoke_chain(movie_name):
    result = chain.invoke({"movie_name": movie_name})
    memory.save_context({"input": movie_name}, {"output": result.content})
    return result.content




In [9]:
invoke_chain("해리포터와 마법사의 돌")

⚡🧙🔮

'⚡🧙🔮'

In [10]:
invoke_chain("토이스토리")

🤠🚀🧸

'🤠🚀🧸'

In [11]:
invoke_chain("내가 앞서 질문했던 영화가 뭔지 다 알려줘")

당연히요! 앞서 질문하신 영화들은 '라이프 오브 파이'는 🐅🌊🚣, '인터스텔라'는 🚀🌌⏰, '기생충'은 🏠💰🕷️, '타이타닉'은 🚢💎💕, '해리포터와 마법사의 돌'은 ⚡🧙🔮, '토이스토리'는 🤠🚀🧸 이렇게요! 부족한 점이 있으면 더 물어봐주세요!The human mentions various movies like '라이프 오브 파이', '인터스텔라', '기생충', '타이타닉', '해리포터와 마법사의 돌', and '토이스토리'. The AI responds with emojis representing each movie and offers to provide more information if needed.

"당연히요! 앞서 질문하신 영화들은 '라이프 오브 파이'는 🐅🌊🚣, '인터스텔라'는 🚀🌌⏰, '기생충'은 🏠💰🕷️, '타이타닉'은 🚢💎💕, '해리포터와 마법사의 돌'은 ⚡🧙🔮, '토이스토리'는 🤠🚀🧸 이렇게요! 부족한 점이 있으면 더 물어봐주세요!"